In [24]:
##크롤링 대상 사이트 : https://www.computer.org (IEEE)

#사용할 라이브러리

import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from multiprocessing import Pool, Array, Process
import time


In [25]:
#url 크롤링하는 함수 정의

def croll_link(next_url,link_name):
    req = requests.get('https://www.computer.org'+next_url)
    soup = bs(req.text, 'html.parser')
    next_r_url = soup.select(link_name)
    
    parsed = [] 
    for i in range(0,len(next_r_url)):
        parsed.append(next_r_url[i].get("href")) #url 추출해서 list에 append
    
    return parsed #list 반환

In [26]:
#크롤링 - site에서는 주제-연도-달(2-3달)-내용 으로 되어있음

start = time.time()

# computer science subject link croll
transactionLinks = croll_link('/csdl','a.transactionLink')

# subject -> volume(year)
volumeLinks = croll_link(transactionLinks[0],'a.volumeLink')

# volume -> issue(months)
issueLinks = croll_link(volumeLinks[0],'a.issueLink')

# issue -> contents
abstractLinks = croll_link(issueLinks[0],'div.col-xs-12 > div > div > div > button > a') # contents

# abstractLinks에서 html만 추출 (javascript와 pdf도 태그가 같았기에 soup로 구분하지 못했음)
new_absLinks = []
for i in range(0,len(abstractLinks)):
    if(abstractLinks[i][-8:] == 'abs.html'):
        new_absLinks.append(abstractLinks[i])
        
close = time.time()

print(close-start)


14.117388010025024


In [27]:
abstractLinks = new_absLinks
new_absLinks = []

#abstractLinks

In [28]:
#contents
req = requests.get('https://www.computer.org'+abstractLinks[1])
soup = bs(req.text, 'html.parser')

In [29]:
#title 크롤링
abs_title = (soup.select('div.abstractTitle > a > h2')[0].text) # title 추출
abs_title

'Simulating the Large-Scale Erosion of Genomic Privacy Over Time'

In [30]:
# url에 date내용이 들어있음
abs_yymm = abstractLinks[1][15:22]
abs_yymm

'2018/05'

In [31]:
#contents 크롤링
abs_contents = soup.select('div.abstractText.abstractTextMB')[0].text
#abs_contents


## Data set 늘리기


In [32]:
# Month를 다 합쳐봄 - volume은 같음
# 다 하면 너무 많을 것 같아서 일단 issue(month)만 합쳤음

total_absLink = []
for i in range(0,len(issueLinks)):
    total_absLink += croll_link(issueLinks[i],'div.col-xs-12 > div > div > div > button > a')

#total_absLink

In [33]:
final_absLink = []
for i in range(0,len(total_absLink)):
    if(total_absLink[i][-8:] == 'abs.html'):
        final_absLink.append(total_absLink[i])
#final_absLink

In [86]:
from datetime import datetime

#파일명을 현재 시간으로
now = str(datetime.now())[0:19]
filename = ("./output/"+"%s"+"_output.txt") % now

start = time.time() # 시간 기록

#크롤링 하는 함수
def croll_abs(link):
    req = requests.get('https://www.computer.org'+link)
    soup = bs(req.text, 'html.parser')
    journal = {}
    journal['title'] = soup.select('div.abstractTitle > a > h2')[0].text
    journal['abstract'] = soup.select('div.abstractText.abstractTextMB')[0].text
    journal['date'] = link[15:22]
  
    #파일을 'add'모드로 열기
    with open(filename, 'a') as f: 
        #각 요소를 tap으로 구분해서 쓰기
        o_data = "%s\t%s\t%s\n" % (journal['date'], journal['title'],journal['abstract'])
        f.write(o_data) 

#프로세스 개수 설정하고 프로세스 돌리기
pool = Pool(processes=6)
p = pool.map(croll_abs,final_absLink)

close = time.time() # 끝나는 시간 기록
print(close-start) # 총 걸린 시간

67.66526794433594


In [89]:
#pandas를 이용하여 output파일을 읽어오기
data = pd.read_csv(filename, sep="\t", header=None)

#column 이름 설정
data.columns = ['Date','Title','Contents']
data

,Date,Title,Contents
0,2018/05,Local-Nearest-Neighbors-Based Feature Weightin...,Selecting functional genes is essential for an...
1,2018/04,Querying of Disparate Association and Interact...,"In biomedical applications, network models are..."
2,2018/05,GenoPri'16: International Workshop on Genome P...,NaN
3,2018/05,Hardware Accelerator for the Multifractal Anal...,The multifractal analysis has allowed to quant...
4,2018/05,Biological Event Trigger Identification with N...,Biological Event Extraction is an important ta...
5,2018/05,Cell Population Tracking in a Honeycomb Struct...,Developing algorithms for plant cell populatio...
6,2018/05,Sequence-Based Prediction of Putative Transcri...,A transcription factor (TF) is a protein that ...
7,2018/05,Heavy-Tailed Noise Suppression and Derivative ...,Most existing array comparative genomic hybrid...
8,2018/04,A Sparse Regression Method for Group-Wise Feat...,The method of Sorted L-One Penalized Estimatio...
9,2018/05,Gene Tree Construction and Correction Using Su...,The supertree problem asking for a tree displa...
